In [158]:
import numpy as np
from skimage import io
import napari
import pandas as pd
import zarr
import os
import pickle
import matplotlib.pyplot as plt 
import cv2
import tiffile as tf
import glob
import pickle

### Set load path and save path

In [166]:
##### change path
# first path need to be first brain,second path need to be another brain
fix_n5_path = ['/mnt/aperto/fused/fused.n5','/mnt/aperto/Tatz_brain_data/fused/fused.n5']

# save path
#create the directory if it does not exist
##### change path
# you need to change to the folder you want
directory = './crops/'
if not os.path.exists(directory):
    os.makedirs(directory)
    print("Directory created:", directory)
##### change path
# you need to change to the folder you want
save_path = directory
##### change path
#you need to change to the folder you want metadata path
meta_path = './info.pkl'

# create Zarr file object
fix_zarr = [zarr.open(store=zarr.N5Store(fix_n5_path[0]), mode='r'), zarr.open(store=zarr.N5Store(fix_n5_path[1]), mode='r')]
# if you use ngff ome.zarr
# mov_zarr_path = '/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/registration/round02.zarr'
# mov_zarr = zarr.open(mov_zarr_path, mode='r')

n5_setups = list(fix_zarr[0].keys())

voxel_size = (2.0,1.3,1.3)

Directory created: ./crops/


In [96]:
# set your parameters here
reference_chan = 3 # Integer or None
segment_chan = 4

# [100,256,256] crop size and FoV [100,768,768] are recommended for the 2D annotation
crop_size = [100,256,256]
FoV = [100,768,768]

# set corner positions. I suggest finding the positions using BigDataViewer or relevant.
# use the fourth column to add information
center_positions = [
    [1235,2510,775, 1],
    [1325,3350,1542,2],
    [1725,4632,1602]
]

corner_positions = [[(i-j//2) for i,j in zip(cent_pos, crop_size + [0])] for cent_pos in center_positions]

# set True for 2D annotation and set False for 3D annotation
select_plane = True

# processing of parameters
if not all([(j-i)>=0 for i,j in zip(crop_size, FoV)]):
    raise ValueError('FoV should be larger than crop_size')

In [97]:
# make metadata file if it does not exist.
if not os.path.exists(meta_path):
    df = pd.DataFrame(columns=['ID', 'integer_ID', 'instance_counts', 'corner', 'source', 'ref_channel', 'channel', 'crop_size', 'isHard', 'plane_position','model_rank', 'comments'])
else:
    df = pd.read_pickle(meta_path)


In [98]:
# if you have trouble like ImportError: cannot import name 'fastCopyAndTranspose' from 'numpy.core.multiarray' 
# try this
#!pip install --upgrade numpy

### Add 4 models prediction

#### load original data and get the image

In [99]:
with open('./annotation_position_info.pkl', 'rb') as f:
    ori_data = pickle.load(f)

/tmp/ipykernel_287315/2941550060.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  ori_data = pickle.load(f)


In [100]:
# find all files in the directory ending with '_img.tif', and '_mask.tif' 
##### change path
#you need to change this path to your image and mask file path
img_dir = '/mnt/aperto/yin/cellpose_training/data/image_masks/' 
img_files = sorted(glob.glob(f'{img_dir}*_img.tif'))
mask_files = sorted(glob.glob(f'{img_dir}*mask.tif'))
img_id = [item.split('/')[-1][:4] for item in img_files]

In [101]:
# check all the id are valid
for item in img_id:
    try:
        ori_data.iloc[int(item)]
    except:
       print(item, 'invalid id')

In [102]:
# optional run, check source of data
set(ori_data['source'])

{'/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/fused/fused.n5',
 '/mnt/ampa02_data01/tmurakami/240425_whole_4color_2nd_M037-3pb/fused/fused.n5'}

In [103]:
# optional run, check source of data
source1 = []
source2 = []
for i in range(len(ori_data)):
    if ori_data['source'][i].split('/')[-3] == '240417_whole_4color_1st_M037-3pb':
        source1.append(i)
    elif ori_data['source'][i].split('/')[-3] == '240425_whole_4color_2nd_M037-3pb':
        source2.append(i)
        

#### load cellpose prediction

In [104]:
# you need to change path to your pkl path
with open('./cellpose_nonorm_all.pkl', 'rb') as f:
    cellpose_data = pickle.load(f)


#### load stardist results

In [105]:
with open('./stardist_all.pkl', 'rb') as f:
    stardist_data = pickle.load(f)

#### load swinUnetr results

In [148]:
with open('./swin2d.pkl', 'rb') as f:
    swin2d_predict = pickle.load(f)

In [149]:
with open('./swin3d_win3.pkl', 'rb') as f:
    swin_3D_pretrain_window3_predict = pickle.load(f)

#### load test file

In [152]:
import json
json_file_path = './test_files.json'

# read test json file
with open(json_file_path) as f:
    test_data = json.load(f)

# # filter norm test data
test_files = [f for f in test_data if '*img.tif' in f]

In [153]:
# Define a function to recursively search for matching strings

import json
import fnmatch
def find_matching_strings(obj, pattern):
    matches = []
    if isinstance(obj, dict):
        for key, value in obj.items():
            matches.extend(find_matching_strings(value, pattern))
    elif isinstance(obj, list):
        for item in obj:
            matches.extend(find_matching_strings(item, pattern))
    elif isinstance(obj, str):
        if fnmatch.fnmatch(obj, pattern):
            matches.append(obj)
    return matches

# Define the pattern to match
pattern = '*img.tif'

# Find all matching strings in the JSON data
matching_strings = find_matching_strings(test_data, pattern)

In [154]:
test_id = [item.split('_')[0] for item in matching_strings]
test_id = sorted(test_id, key=int)

#### load shuffled list

In [155]:
with open('./shuffle_model.pkl', 'rb') as f:
    shuffle_model = pickle.load(f)

In [156]:
model_rank_list = {}

In [164]:
#### img_id
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

### Open napari

In [168]:
for ori_id in test_id:
    ori_corner_position = [ori_data['corner'][int(ori_id)]]
    ori_plane = int(ori_data['plane_position'][int(ori_id)])
    segment_chan = int(ori_data['channel'][int(ori_id)])
    reference_chan =  int(ori_data['ref_channel'][int(ori_id)])
    #check which source it should be int 0 or 1
    brain_source = int(ori_data['source'][int(ori_id)].split('/')[-3].split('_')[-2][0])-1
    ##########
    ## add cellpose prediction
    try:
        cellpose_predict = cellpose_data[ori_id]
    except:
        print(f'this id {ori_id} is exceed original id range, continue')
        continue
    model_id = shuffle_model[ori_id]
    
    # expand cellpose_prediction to correct size
    # create a 256x256 array of zeros
    single_layer = np.zeros((768, 768))
    # stack this array 100 times along a new axis (axis=0)
    data1 = np.stack([single_layer] * 100, axis=0)
    expanded_array = np.zeros((768, 768))
    # Calculate the starting indices to place the 256x256 array in the center
    start_index = (768 - 256) // 2
    # Place the 256x256 array in the center of the 768x768 array
    expanded_array[start_index:start_index+256, start_index:start_index+256] = cellpose_predict
    data1[ori_plane] = expanded_array
    data1 = data1.astype(int)


    

    # create a 256x256 array of zeros
    single_layer = np.zeros((768, 768))
    # stack this array 100 times along a new axis (axis=0)
    data2 = np.stack([single_layer] * 100, axis=0)
    expanded_array = np.zeros((768, 768))
    # Calculate the starting indices to place the 256x256 array in the center
    start_index = (768 - 256) // 2
    # Place the 256x256 array in the center of the 768x768 array
    expanded_array[start_index:start_index+256, start_index:start_index+256] = stardist_data[ori_id]
    data2[ori_plane] = expanded_array
    data2 = data2.astype(int)


    # create a 256x256 array of zeros
    single_layer = np.zeros((768, 768))
    # stack this array 100 times along a new axis (axis=0)
    data3 = np.stack([single_layer] * 100, axis=0)
    expanded_array = np.zeros((768, 768))
    # Calculate the starting indices to place the 256x256 array in the center
    start_index = (768 - 256) // 2
    # Place the 256x256 array in the center of the 768x768 array
    expanded_array[start_index:start_index+256, start_index:start_index+256] = swin2d_predict[ori_id]
    data3[ori_plane] = expanded_array
    data3 = data3.astype(int)


    # create a 256x256 array of zeros
    single_layer = np.zeros((768, 768))
    # stack this array 100 times along a new axis (axis=0)
    data4 = np.stack([single_layer] * 100, axis=0)
    expanded_array = np.zeros((768, 768))
    # Calculate the starting indices to place the 256x256 array in the center
    start_index = (768 - 256) // 2
    # Place the 256x256 array in the center of the 768x768 array
    expanded_array[start_index:start_index+256, start_index:start_index+256] = swin_3D_pretrain_window3_predict[ori_id]
    data4[ori_plane] = expanded_array
    data4 = data4.astype(int)

    
    data_map = {
        1: data1,
        2: data2,
        3: data3,
    }
    
    class1_data = data_map.get( model_id[0], data4)     
    class2_data = data_map.get( model_id[1], data4)
    class3_data = data_map.get( model_id[2], data4)     
    class4_data = data_map.get( model_id[3], data4)     
    
    pos = ori_corner_position[0]
    if len(pos) <= 2:
        raise ValueError('The position should have length 3')
    elif len(pos) == 3:
        isHard = 0
    elif len(pos) == 4:
        isHard = pos[-1]
        pos = pos[:-1]
    else:
        raise ValueError('You have a wrong position format')
        
    print(f"The index {ori_id} with the position {pos}")
    idx = int(ori_id)
    
    # find out any duplication between the current data and the metadata
    # if it is duplicated, ask 
    flag = False
    if df['corner'].isin([pos]).any():
        for k in df['integer_ID'][df['corner'].isin([pos])].to_list():
            if ((df.loc[k,'source'] == fix_n5_path[brain_source]) and 
                (df.loc[k,'ref_channel'] == reference_chan) and 
                (df.loc[k,'channel'] == segment_chan) and 
                (df.loc[k,'crop_size'] == crop_size) and
                (df.loc[k,'select_plane'] == select_plane)):
                flag = True
                idx = k
    
    if flag:
        ans = input("Do you want to re-analyze the data? y or n")
        if ans != 'y':
            continue
        
    # set file path to be saved for both image and mask
    prefix = str(idx)
    while len(prefix) < 4:
        prefix = '0' + prefix
    img_path = os.path.join(save_path, prefix+'_img.tif')
    mask_path = [os.path.join(save_path, prefix+'_model1.tif')
                ,os.path.join(save_path, prefix+'_model2.tif')
                ,os.path.join(save_path, prefix+'_model3.tif')
                ,os.path.join(save_path, prefix+'_model4.tif')]

    #### get which sources

    # get the image of a channel to be segmented
    FoV_stack = []
    img = fix_zarr[brain_source][n5_setups[segment_chan]]['timepoint0']['s0']

    # set the corner of FoV in napari
    top_corner = tuple(i-(k-j)//2 for i,j,k in zip(pos, crop_size, FoV))
    bottom_corner = tuple(i+j+(k-j)//2 for i,j,k in zip(pos, crop_size, FoV))
    top_corner = tuple(j if j>=i else i for i,j in zip([0,0,0],top_corner))
    bottom_corner = tuple(j if j<=i else i for i,j in zip(img.shape,bottom_corner))
    
    # prepare to make border lines
    top_border_corner = tuple((k-j)//2 for j,k in zip(crop_size, FoV))
    bottom_border_corner = tuple(j+(k-j)//2 for j,k in zip(crop_size, FoV))
    
    FoV_segment = img[tuple(slice(i,j) for i,j in zip(top_corner, bottom_corner))]

    # get the image of a reference of channel
    if reference_chan is not None:
        img = fix_zarr[brain_source][n5_setups[reference_chan]]['timepoint0']['s0']
        FoV_reference = img[tuple(slice(i,j) for i,j in zip(top_corner, bottom_corner))]
        FoV_stack.append(FoV_reference)
    
    FoV_stack.append(FoV_segment)
    FoV_stack = np.stack(FoV_stack)

    ##### FoV is a 2channel(reference and signal) Field of view
    # open Napari. Pause for loop until close the window
    viewer = napari.Viewer()
    # set hotkey
     # set quit key
    @viewer.bind_key('q')
    def close_viewer(viewer):
        print("Closing viewer...")
        viewer.close()

     #set hide hotkey
    @viewer.bind_key('h')
    def toggle_layer_visibility(viewer):
        layer = viewer.layers.selection.active
        if layer is not None:
            layer.visible = not layer.visible
    
    viewer.add_image(FoV_stack, channel_axis=0, scale=voxel_size, contrast_limits=[0,65535])
    viewer.add_shapes([[bottom_border_corner[1]*voxel_size[1],bottom_border_corner[2]*voxel_size[2]],[top_border_corner[1]*voxel_size[1],bottom_border_corner[2]*voxel_size[2]]],
                      edge_width=2,edge_color='white',ndim=2,shape_type='line')
    viewer.add_shapes([[top_border_corner[1]*voxel_size[1],bottom_border_corner[2]*voxel_size[2]],[top_border_corner[1]*voxel_size[1],top_border_corner[2]*voxel_size[2]]],
                  edge_width=2,edge_color='white',ndim=2,shape_type='line')
    viewer.add_shapes([[bottom_border_corner[1]*voxel_size[1],bottom_border_corner[2]*voxel_size[2]],[bottom_border_corner[1]*voxel_size[1],top_border_corner[2]*voxel_size[2]]],
                  edge_width=2,edge_color='white',ndim=2,shape_type='line')
    viewer.add_shapes([[bottom_border_corner[1]*voxel_size[1],top_border_corner[2]*voxel_size[2]],[top_border_corner[1]*voxel_size[1],top_border_corner[2]*voxel_size[2]]],
                  edge_width=2,edge_color='white',ndim=2,shape_type='line')
    
    #labels = viewer.add_labels(np.zeros_like(FoV_segment), name='segmentation', scale=voxel_size)
    #add three classes label layers
    labels_class1 = viewer.add_labels(class1_data,  name=f'model1({ori_plane})', scale=voxel_size)
    labels_class1.opacity = 1.0
    labels_class1.brush_size = 1
    labels_class2 = viewer.add_labels(class2_data,  name=f'model2({ori_plane})', scale=voxel_size)
    labels_class2.opacity = 1.0
    labels_class2.brush_size = 1
    labels_class3 = viewer.add_labels(class3_data,  name=f'model3({ori_plane})', scale=voxel_size)
    labels_class3.opacity = 1.0
    labels_class3.brush_size = 1
    labels_class4 = viewer.add_labels(class4_data,  name=f'model4({ori_plane})', scale=voxel_size)
    labels_class4.opacity = 1.0
    labels_class4.brush_size = 1
    
    viewer.camera.zoom = 1.5
    
    viewer.dims.current_step = (ori_plane,400,400)
    viewer.show(block=True)
    model_rank = input("Please type in the rank of model")
    model_rank_list[ori_id]= model_rank
    comment = input("Do you have any comments when ranking the model?")

    
    sub_area_slicer = tuple(slice(i,j) for i,j in zip(top_border_corner,bottom_border_corner))
    ######
    # subarea shape
    # save images and segmentation.
    img = np.swapaxes(FoV_stack[(slice(0,None),)+sub_area_slicer],0,1)
    ######
    # modify this to cellpose label
    #labels_img = labels.data[sub_area_slicer]
    labels_img_class1 = labels_class1.data[sub_area_slicer]
    labels_img_class2 = labels_class2.data[sub_area_slicer]
    labels_img_class3 = labels_class3.data[sub_area_slicer]
    labels_img_class4 = labels_class4.data[sub_area_slicer]

   
    if select_plane:
        print('selected and save')
        ##########
        # modify this to original label
        plane_pos = np.argmax((labels_class1.data>0).sum(axis=(1,2)))
        img = img[plane_pos,...]
        labels_img_class1 = labels_img_class1[plane_pos,...]
        labels_img_class2 = labels_img_class2[plane_pos,...]
        labels_img_class3 = labels_img_class3[plane_pos,...]
        labels_img_class4 = labels_img_class4[plane_pos,...]
        
        io.imsave(img_path, img, plugin='tifffile', imagej=True, metadata={'axes': 'CYX'})
        io.imsave(mask_path[0], labels_img_class1, plugin='tifffile')
        io.imsave(mask_path[1], labels_img_class2, plugin='tifffile')
        io.imsave(mask_path[2], labels_img_class3, plugin='tifffile')
        io.imsave(mask_path[3], labels_img_class4, plugin='tifffile')

    else:
        print('#############')
        print('not selected and save')
        io.imsave(img_path, img, plugin='tifffile', imagej=True, metadata={'axes': 'ZCYX'})
        io.imsave(mask_path[0], labels_img_class1, plugin='tifffile')
        io.imsave(mask_path[1], labels_img_class2, plugin='tifffile')
        io.imsave(mask_path[2], labels_img_class3, plugin='tifffile')
    # update the metadata
    df.loc[idx,'ID'] = prefix
    df.loc[idx,'integer_ID'] = idx
    count = (np.unique(labels_img_class1)).size - 1
    df.loc[idx,'instance_counts'] = count
    df.at[idx,'corner'] = pos
    df.loc[idx, 'source'] = fix_n5_path[brain_source]
    df.loc[idx, 'ref_channel'] = reference_chan
    df.loc[idx, 'channel'] = segment_chan
    df.at[idx, 'crop_size'] = crop_size
    df.loc[idx, 'select_plane'] = select_plane
    df.loc[idx, 'isHard'] = isHard
    df.loc[idx, 'model_rank'] = model_rank
    df.loc[idx, 'comments'] = comment
    
    if select_plane:
        df.loc[idx, 'plane_position'] = int(plane_pos)
    else:
        df.loc[idx, 'plane_position'] = -1



The index 0005 with the position [1227, 551, 2984]


Do you want to re-analyze the data? y or n y


Closing viewer...


AttributeError: 'Window' object has no attribute '_qt_window'

### Save results

In [124]:
# save the file through pickle the metadata
df.to_pickle(meta_path)

#### set begining id for next round

In [163]:
# if you don't want to run all again, set the beginning id as begin
curent_id = test_id.index(ori_id)
begin_id = curent_id
print(f'current image id {curent_id}, next begin id {begin_id}')

current image id 37, next begin id 37


In [126]:
# (optional)if you want to start all over again, reset begin_id as 0
# begin_id = 0